Importing All Library

In [ ]:
!pip install pywavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.8 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import cv2
import pywt
from skimage.feature import hog
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import  ModelCheckpoint
from tensorflow.keras.layers import GlobalAveragePooling2D
import gdown
import zipfile
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Add, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import numpy as np

Eye Disease Datasets

In [ ]:
# Google Drive file URL
file_url = "https://drive.google.com/uc?id=1vHCvd9ZFkY9lfNwUnnMUbDQZjjpby8lv"

# Specify the output file path (the path where you want to save the file)
output_file = "disease.zip"  # Replace with actual file name

# Download the file
gdown.download(file_url, output_file, quiet=False)

zip_file = "disease.zip"  # Name of the downloaded file

# Make sure output directory exists
output_dir = "/content/extracted_file"
os.makedirs(output_dir, exist_ok=True)

# Extract the contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F
From (redirected): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F&confirm=t&uuid=50b70997-4c3a-4b52-9724-ae6b50621c62
To: /content/disease.zip
100%|██████████| 3.85G/3.85G [00:23<00:00, 164MB/s]


Teacher Model Weights

In [ ]:
# Google Drive file URL
file_url = "https://drive.google.com/uc?id=195vi2HWF4MHn1_y_uFYFNwq_bTNBI1ik"

# Specify the output file path (the path where you want to save the file)
output_file = "/content/vit_model.keras"  # Replace with actual file name

# Download the file
gdown.download(file_url, output_file, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=195vi2HWF4MHn1_y_uFYFNwq_bTNBI1ik
From (redirected): https://drive.google.com/uc?id=195vi2HWF4MHn1_y_uFYFNwq_bTNBI1ik&confirm=t&uuid=2a5ad217-45a6-45b1-99e4-10ddd62a6cf4
To: /content/vit_model.keras
100%|██████████| 574M/574M [00:02<00:00, 199MB/s]


'/content/vit_model.keras'

All Individual Models

In [ ]:
# Google Drive file URL
file_url = "https://drive.google.com/uc?id=1vHvi7eIdzNBJbsnE68ktIkJnRZLiaq1N"

# Specify the output file path (the path where you want to save the file)
output_file = "individual.zip"  # Replace with actual file name

# Download the file
gdown.download(file_url, output_file, quiet=False)

zip_file = "individual.zip"  # Name of the downloaded file

# Make sure output directory exists
output_dir = "/content/individual"
os.makedirs(output_dir, exist_ok=True)

# Extract the contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)


Downloading...
From (original): https://drive.google.com/uc?id=1vHvi7eIdzNBJbsnE68ktIkJnRZLiaq1N
From (redirected): https://drive.google.com/uc?id=1vHvi7eIdzNBJbsnE68ktIkJnRZLiaq1N&confirm=t&uuid=fee1b349-0853-4bac-8f72-82ca20657a95
To: /content/individual.zip
100%|██████████| 114M/114M [00:01<00:00, 80.7MB/s]


Teacher Model Architecture and loading its weights

In [ ]:
# Patch & Position Embedding Layer
class PatchEmbedding(tf.keras.layers.Layer):
    def __init__(self, patch_size, projection_dim):
        super().__init__()
        self.patch_size = patch_size
        self.projection_dim = projection_dim
        self.projection = tf.keras.layers.Conv2D(filters=projection_dim,
                                                 kernel_size=patch_size,
                                                 strides=patch_size,
                                                 padding='valid')
        self.flatten = tf.keras.layers.Reshape((-1, projection_dim))

    def call(self, images):
        x = self.projection(images)
        x = self.flatten(x)
        return x

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.pos_embedding = self.add_weight(
            name="pos_embed",
            shape=(1, num_patches + 1, projection_dim),
            initializer="random_normal",
            trainable=True
        )
        self.cls_token = self.add_weight(
            name="cls_token",
            shape=(1, 1, projection_dim),
            initializer="random_normal",
            trainable=True
        )

    def call(self, x):
        batch_size = tf.shape(x)[0]
        cls_tokens = tf.broadcast_to(self.cls_token, [batch_size, 1, tf.shape(x)[-1]])
        x = tf.concat([cls_tokens, x], axis=1)
        return x + self.pos_embedding


# Transformer Encoder Block
def transformer_encoder(x, projection_dim, num_heads, mlp_dim, dropout_rate):
    x1 = LayerNormalization(epsilon=1e-6)(x)
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1)
    x2 = Add()([x, attention_output])
    x3 = LayerNormalization(epsilon=1e-6)(x2)
    x3 = Dense(mlp_dim, activation='gelu')(x3)
    x3 = Dropout(dropout_rate)(x3)
    x3 = Dense(projection_dim)(x3)
    x3 = Dropout(dropout_rate)(x3)
    return Add()([x2, x3])

# Build ViT
def build_vit(input_shape=(224, 224, 3), num_classes=7,
              patch_size=16, projection_dim=256, transformer_layers=12,
              num_heads=8, mlp_dim=512, dropout_rate=0.1):

    inputs = Input(shape=input_shape)

    # Patch Embedding + Positional Encoding
    patch_embed = PatchEmbedding(patch_size, projection_dim)(inputs)
    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)
    encoded = PositionalEncoding(num_patches, projection_dim)(patch_embed)

    # Transformer Encoder
    for _ in range(transformer_layers):
        encoded = transformer_encoder(encoded, projection_dim, num_heads, mlp_dim, dropout_rate)

    # Classification head
    x = LayerNormalization(epsilon=1e-6)(encoded)
    x = x[:, 0, :]  # CLS token
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)

    return Model(inputs, outputs)



# Image size, batch size
img_size = (224, 224)
batch_size = 32

# Rebuild the model with the same parameters
combine_vit = build_vit(
    input_shape=(224, 224, 3),
    num_classes=7,
    patch_size=16,
    projection_dim=256,
    transformer_layers=12,
    num_heads=8,
    mlp_dim=512,
    dropout_rate=0.1
)

combine_vit.compile(optimizer=Adam(1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                  metrics=['accuracy'])

combine_vit.load_weights("/content/vit_model.keras")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 406 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Student Model Architecture

In [ ]:
def build_ImprovedCNN():
    model = tf.keras.Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        GlobalAveragePooling2D(),

        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(7, activation='softmax')  # Assuming you're using a 3-class output
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

student = build_ImprovedCNN()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Making Traing dataloader

In [ ]:
data_path = "/content/extracted_file/Retinal Fundus Images - Copy/train"

# Datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.1,
    subset='training',
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

Found 55446 files belonging to 7 classes.
Using 49902 files for training.


Knowledge Distillation Function

In [ ]:
def distillation_loss(y_true, y_student_logits, y_teacher_logits, temperature=4.0, alpha=0.7):
    # Hard loss (true labels)
    hard_loss = tf.keras.losses.categorical_crossentropy(y_true, y_student_logits, from_logits=True)

    # Softened probabilities
    soft_teacher = tf.nn.softmax(y_teacher_logits / temperature)
    soft_student = tf.nn.log_softmax(y_student_logits / temperature)

    # KL divergence
    kl_loss = tf.reduce_mean(tf.reduce_sum(soft_teacher * (tf.math.log(soft_teacher + 1e-10) - soft_student), axis=1))
    soft_loss = kl_loss * (temperature ** 2)

    return alpha * soft_loss + (1. - alpha) * tf.reduce_mean(hard_loss)

Importing Individual Model and making its model to index

In [ ]:
# Load models
AMD = load_model('/content/individual/Individual_disease/AMD.keras')
Cataract = load_model('/content/individual/Individual_disease/Cataract.keras')
DR = load_model('/content/individual/Individual_disease/DR.keras')
Glaucoma = load_model('/content/individual/Individual_disease/Glaucoma.keras')
Hypertensive_Retinopathy = load_model('/content/individual/Individual_disease/Hypertensive_Retinopathy.keras')
Pathological_Myopia = load_model('/content/individual/Individual_disease/Pathological_Myopia.keras')

# Mapping models directly to class indices (as per your combined model)
model_to_index = {
    AMD: 0,
    Cataract: 1,
    DR: 2,
    Glaucoma: 3,
    Hypertensive_Retinopathy: 4,
    Pathological_Myopia: 6,
}


Creating Teacher soft logits using combine and individual models

In [ ]:
def create_teacher_soft_labels(image, model_to_index, combined_model):
    num_classes = 7  # 6 diseases + 1 normal

    # Combined model logits
    combined_logits = combined_model.predict(image)[0]  # shape: (7,)

    # Initialize soft logits
    soft_logits = np.zeros(num_classes, dtype=np.float32)

    # Weighting
    combined_weight = 0.45
    individual_weight = 0.55

    # Add combined model prediction
    soft_logits += combined_logits * combined_weight
    soft_logits[5] += combined_logits[5] * 0.55 # doing this for normal fundus

    # Add individual model predictions
    for model, index in model_to_index.items():
        pred = model.predict(image)[0]  # shape: (2,)
        prob_disease = tf.nn.softmax(pred)[0].numpy()  # Assuming [disease, not disease]
        soft_logits[index] += prob_disease * individual_weight

    # Normalize logits to probabilities
    soft_probs = tf.nn.softmax(soft_logits).numpy()
    return soft_probs


Training Student Model

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
combine_vit.trainable = False  # Freeze the teacher model


for epoch in range(1):
    # print(f"Epoch {epoch+1}/{1}")

    for step, (images, labels) in enumerate(train_ds):  # train_ds is a tf.data.Dataset
        with tf.GradientTape() as tape:
            # Forward pass
            student_logits = student(images, training=True)
            teacher_logits = create_teacher_soft_labels(images, model_to_index,combine_vit)

            # Compute distillation loss
            loss = distillation_loss(labels, student_logits, teacher_logits)

        # Compute gradients and update student model
        grads = tape.gradient(loss, student.trainable_variables)
        optimizer.apply_gradients(zip(grads, student.trainable_variables))

        if step % 100 == 0:
          print(f"Step {step}, Loss: {loss.numpy().mean():.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:666: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Step 0, Loss: 0.8080
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
student.save("/content/stundent_model.keras")
